In [ ]:
import pandas as pd
import re
from datetime import datetime

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/data/news_july.csv")

### **데이터 합치기**

In [ ]:
eco = pd.read_csv("/content/drive/MyDrive/data/news_에코프로_7월.csv")
cell = pd.read_csv("/content/drive/MyDrive/data/news_2차전지_5_to_7.csv")

In [ ]:
# 2차전지 - 7월 데이터만 추출
july_data = cell[cell['작성일시'].str.contains('2023년 07월')]
july_data.head()

In [ ]:
# 에코프로 & 이차전지 데이터프레임 합치기
data = pd.concat([eco,july_data], ignore_index=True)
data = data.drop('Unnamed: 0', axis=1)
data.head()

In [ ]:
#data.to_csv("/content/drive/MyDrive/data/news_july.csv",index = None)
#data.to_excel("/content/drive/MyDrive/data/news_july.xlsx",index = None)

### **결측치 제거**

In [ ]:
# 결측치 확인
data.isnull().sum()

언론사     0
제목      0
작성일시    0
본문      3
링크      0
dtype: int64

In [ ]:
# 결측치 제거
data = data.dropna(axis=0)

In [ ]:
data.isnull().sum()

언론사     0
제목      0
작성일시    0
본문      0
링크      0
dtype: int64

### **중복 데이터 제거**

In [ ]:
# 중복 데이터 확인
data[data.duplicated(['링크'])].head()

,언론사,제목,작성일시,본문,링크
11,매일경제,에코프로 무한질주에 증권가 ‘곤혹’…“에코프로비엠은 더 올라”,2023년 07월 14일,지난 10일 오후 서울 종로구 연합인포맥스 스크린에 에코프로 차트가 띄워져 있다....,https://n.news.naver.com/mnews/article/009/000...
13,서울경제,어닝쇼크에도 98만원 지킨 에코프로…外人 이달 3600억 샀다,2023년 07월 15일,2분기 실적발표에도 단단한 주가 황제주 목전숏커버링 영향? 외인 삼전 이어 순매수 ...,https://n.news.naver.com/mnews/article/011/000...
18,헤럴드경제,"“킹코프로는 실적 부진해도, 증권사 손 놔도 올라요”…에코프로의 ‘미스터리 주가’ ...",2023년 07월 15일,[게티이미지뱅크][헤럴드경제=권제인 기자] 코스피가 한 주간 100포인트 넘게 상승...,https://n.news.naver.com/mnews/article/016/000...
80,매일경제,"“드디어 뚫었다”…에코프로, 100만원 돌파하며 황제주 등극",2023년 07월 10일,에코프로. [사진 제공 = 연합뉴스]에코프로가 10일 장 초반 100만원을 넘어서...,https://n.news.naver.com/mnews/article/009/000...
81,국민일보,"‘기이한 열풍’ 에코프로, 극성개미 지지에 ‘황제주’ 등극",2023년 07월 11일,96만원대 마감… 올해 777% 상승전기차·2차전지 성장세 힘입어비정상적 열풍… 증...,https://n.news.naver.com/mnews/article/005/000...


In [ ]:
# 중복 데이터 제거
data = data.drop_duplicates(['링크'], keep = 'first')

# 제목만 다르고, 본문 내용은 같은 기사 제거 (같은 날 작성)
data = data.drop_duplicates(['본문'], keep = 'first')

In [ ]:
# 제목과 본문에 검색어 포함안된 기사 확인
print(data[~data['본문'].str.contains('2차 전지|2차전지|에코프로|이차전지')].index)
data[~data['본문'].str.contains('2차 전지|2차전지|에코프로|이차전지')]

In [ ]:
# 본문에 사진 설명만 또는 아무 내용이 없는 기사 + 무관한 기사 제거
data = data[~data['본문'].str.contains('전지훈련')]

data = data[~data['제목'].str.contains('헤드라인')]
data = data[~data['본문'].str.contains('헤드라인')]

data = data[(data['본문'].str.contains('2차 전지|2차전지|에코프로|이차전지') | data['제목'].str.contains('2차 전지|2차전지|에코프로|이차전지'))]

delte_rows = [419,  461, 1455, 1476, 1663, 1940, 1949, 1951, 2013,
            3129, 3319, 3364, 3365, 4503, 4542, 4780, 5028, 5138]
data.drop(delte_rows, axis=0, inplace=True)

### **개행 문자 등 제거**

In [ ]:
def cleaning(text) :
  text = str(text)

  pattern = r"▲ 에코프로 포항사업장\.[^\|]+|에코프로 제공\|+|에코프로 자료사진\|+|에코프로 홈페이지\|+|이메일 : jebo@cbs.co.kr|카카오톡 : @노컷뉴스사이트 : https://url.kr/b71afn|SBS Biz는 여러분의 제보를 기다리고 있습니다.홈페이지 = https://url.kr/9pghjn|.■ 제보하기▷ 전화 : 02-781-1234, 4444▷ 이메일 : kbs1234@kbs.co.kr▷ 카카오톡 : 'KBS제보' 검색, 채널 추가▷ 네이버, 유튜브에서 KBS뉴스를 구독해주세요!|MBC 뉴스는 24시간 여러분의 제보를 기다립니다. ▷ 전화 02-784-4000▷ 이메일 mbcjebo@mbc.co.kr▷ 카카오톡 @mbc제보|/사진제공=네이버증권|/사진=|한경DB|당신의 제보가 뉴스로 만들어집니다"
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  pattern = '<[^>]*>'         # HTML 태그 제거
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  pattern = r'[\n\t]'    # 개행 문자, 탭 문자 제거
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  text = text.replace('↓', '하락')
  text = text.replace('↑', '상승')
  text = text.replace('→', '에서')
  text = text.replace('%', '퍼센트')
  text = text.replace('…', '.')
  text = text.replace("2차전지",'이차전지')

  pattern = r'\([^)]{1,50}\)|\[[^\]]{1,50}\]|<[^>]{1,50}>' # 대개 기자 이름, 사진 설명 등
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  #pattern = r'(\d{4}\.\d{2}\.\d{2}\.)|(\d{4}\.\d{2}\.\d{2})' # 날짜 제거
  #text = re.sub(pattern = pattern, repl = ' ', string = text)

  #pattern = r'[^\uAC00-\uD7A3 0-9a-zA-Z\s.,]'  # 한글, 숫자, 영어, 온점, 쉼 제외 모두 제거 (수치 데이터가 많아서 온점 제외 X)
  pattern = r'[^가-힣ㄱ-ㅎㅏ-ㅣ]'               # 한글 제외 모두 제거
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  pattern = r'\b\w{2,5}\s?(?:기자|기자입니다)\b'   # 기자 이름 제거
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  pattern = '|'.join(data['언론사'])
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  #text= re.sub(r'[^0-9.%]', '', string = text)  # 수치 제외 온점 제거

  pattern = r'\s+'      # 연속 공백 제거
  text = re.sub(pattern = pattern, repl = ' ', string = text)

  return text

In [ ]:
data['제목'] = data['제목'].apply(cleaning)
data['본문'] = data['본문'].apply(cleaning)

### **날짜 형식 변환**

In [ ]:
def convert_date_format(date_str):
    # 기존 형식의 날짜 문자열을 datetime 객체로 변환
    date_obj = datetime.strptime(date_str, '%Y년 %m월 %d일')
    # 원하는 형식의 날짜 문자열로 변환
    new_date_str = date_obj.strftime('%Y-%m-%d')
    return new_date_str

data['작성일시'] = data['작성일시'].apply(convert_date_format)
data = data.rename(columns={'작성일시': '게시일'})

In [ ]:
# 필요없는 열 제거
data = data.drop(['언론사','링크'], axis=1)

In [ ]:
data['본문'][0]

'삼성바이오로직스 목표주가 만원 가장 유력 일 서울 중구 하나은행 딜링룸 직원 모니터에 북한 우주 발사체 발사 관련 뉴스가 나오고 있다 이차전지 업체 에코프로의 주가가 만원을 넘나들며 황제주 등극이 임박한 가운데 제 의 에코프로에 도전하는 다음 후보군에 관심이 쏠리고 있다 일 한국거래소에 따르면 코스닥 시장에서 에코프로는 지난 일 주가가 만원을 돌파하며 년만에 만원대 주식 이른바 황제주 대열에 올라섰다 증권가에서는 에코프로 다음으로 화제주에 등극할 가장 유력한 후보로 삼성바이오로직스를 꼽았다 금융정보업체 에프엔가이드가 최근 개월 동안 증권사들의 목표주가를 취합한 결과 삼성바이오로직스가 만원으로 가장 높게 나타났다 삼성바이오로직스의 현재 주가는 만 원인데 앞으로 퍼센트 가량 상승 여력이 있다고 본 것이다 하나증권은 최근 리포트에서 삼성바이오로직스 목표주가를 만원으로 올렸다 이는 최근 개월 내 제시된 증권사들의 목표주가 중 가장 높은 수치다 하나증권은 삼성바이오로직스의 올해 연간 영업이익이 조 억원을 기록해 지난해보다 약 퍼센트 증가할 것으로 보고 목표주가를 조정했다 박재경 하나증권 연구원은 상업 생산부터 매출 인식이 가능한 국제회계기준 에 따라 공장 실적은 분기부터 반영될 예정인데 성능평가생산 배치가 상업 생산 물량과 함께 매출로 인식돼 초기 가동률이 빠르게 올라올 전망 이라고 말했다 이베스트투자증권도 삼성바이오로직스의 목표주가로 만원을 제시했다 강하나 이베스트투자증권 연구원은 휴미라 바이오시밀러 시장에서 가장 경쟁력을 가진 의약품인 하드리마의 보험등재 여부는 년에 확실해지면서 외형 성장을 이끌 것 이라고 설명했다 증권사들은 삼성바이오로직스가 올해뿐 아니라 내년 실적 전망도 긍정적일 것이라고 전망했다 자회사인 삼성바이오에피스가 전 세계에서 매출이 가장 많은 의약품으로 알려진 휴미라 의 바이오시밀러 시장 진출에 오랜 공을 들여왔는데 그 성과가 내년께 본격화할 것으로 기대되고 있다 삼성바이오로직스에 이어 증권사들의 목표주가가 두 번째로 높은 기업은 화학이다 최근 개

###**결과**

In [ ]:
data.head()

,제목,게시일,본문
0,만원 임박한 에코프로 다음 황제주 후보는 누구,2023-07-15,삼성바이오로직스 목표주가 만원 가장 유력 일 서울 중구 하나은행 딜링룸 직원 모니터...
1,에코프로도 당했다 미즈호 등 불법 공매도 무더기 적발,2023-07-14,주식 빌리지 않고 선매도 하는 무차입 공매도 혐의 회사측 고의 없었다 착오였다 과징...
2,에코프로 무한질주에 증권가 곤혹 에코프로비엠은 더 올라,2023-07-14,지난 일 오후 서울 종로구 연합인포맥스 스크린에 에코프로 차트가 띄워져 있다 코스...
3,하림지주 에코프로 등 상반기 주식관련사채 행사액 조 억,2023-07-14,한국예탁결제원은 올해 상반기 전환사채 교환사채 신주인수권부사채 등 주식관련사채 행...
4,에코프로 포항에 조 투자 양극재 허브 구축,2023-07-13,에코프로가 포항시 남구 블루밸리 국가산업단지에 양극소재 생태계 조성을 위해 조원을...


In [ ]:
data.to_csv("/content/drive/MyDrive/data/news_july_preprocessed.csv")